In [ ]:
import cv2
import vlc
import numpy as np
from keras.models import load_model

# Configuración de la cámara
video_capture = cv2.VideoCapture(0)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

# Cargamos los clasificadores de OpenCV para detectar ojos
eyeLeft = cv2.CascadeClassifier('/Users/Juniot/anaconda3/envs/nombre/Lib/site-packages/cv2/data/haarcascade_lefteye_2splits.xml')  # modelo ojo izquierdo
eyeRight = cv2.CascadeClassifier('/Users/Juniot/anaconda3/envs/nombre/Lib/site-packages/cv2/data/haarcascade_righteye_2splits.xml')  # modelo ojo derecho

# Inicialización del modelo Keras
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)  # inicializar la variable data
model = load_model('keras_model.h5')  # cargar el modelo

# Configuración del reproductor VLC
p = vlc.MediaPlayer("Mejores Sonidos FUERTES para ALARMA Te despiertas SI o SI  tonos para despertador  tonos de alarma.mp3")

# Inicialización de las coordenadas de los ojos
left_x, left_y, left_w, left_h = 0, 0, 0, 0  # coordenadas ojo izquierdo
right_x, right_y, right_w, right_h = 0, 0, 0, 0  # coordenadas ojo derecho
contador = 0  # contador para seguimiento de dormido/despierto

# Bucle principal de captura de video
while True:
    ret, frame = video_capture.read()
    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convertir imagen a escala de grises

    # Mostrar el contador en pantalla
    cv2.rectangle(frame, (0, 0), (width, int(height*0.1)), (0, 0, 0), -1)
    cv2.putText(frame, '             ' + str(contador), (int(width*0.65), int(height*0.08)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

    # Detección del ojo derecho
    ojo_der = eyeRight.detectMultiScale(
        gray,
        scaleFactor=1.15,
        minNeighbors=3,
        minSize=(30, 30)
    )
    for (x, y, w, h) in ojo_der:  # recorrer las detecciones del ojo derecho
        right_x, right_y, right_w, right_h = x, y, w, h
        break  # tomar solo la primera detección

    # Detección del ojo izquierdo
    ojo_izq = eyeLeft.detectMultiScale(
        gray,
        scaleFactor=1.15,
        minNeighbors=3,
        minSize=(30, 30)
    )
    for (x, y, w, h) in ojo_izq:  # recorrer las detecciones del ojo izquierdo
        left_x, left_y, left_w, left_h = x, y, w, h
        break

    # Definir las coordenadas para extraer los ojos
    if left_x > right_x:
        start_x, end_x = right_x, (left_x + left_w)
    else:
        start_x, end_x = left_x, (right_x + right_w)

    if left_y > right_y:
        start_y, end_y = right_y, (left_y + left_h)
    else:
        start_y, end_y = left_y, (right_y + right_h)

    # Comprobación de las dimensiones de la imagen de los ojos
    if (end_x - start_x) > 120 and (end_y - start_y) < 200:
        start_x, start_y, end_x, end_y = start_x - 30, start_y - 50, end_x + 30, end_y + 50
        cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)

        # Extraer y preparar la imagen para el modelo
        img = frame[start_y:end_y, start_x:end_x]
        imagen = cv2.resize(img, (224, 224))
        imagen_normalizada = (imagen.astype(np.float32) / 127.0) - 1
        data[0] = imagen_normalizada

        # Hacer predicción con el modelo
        prediction = model.predict(data)

        # Si la probabilidad de dormido es alta
        if list(prediction)[0][1] >= 0.95:
            cv2.putText(frame, 'Dormido: ' + 'probabilidad: ' + str(round(list(prediction)[0][1], 3)), (10, int(height*0.08)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
            contador += 1

        # Si la probabilidad de despierto es alta
        if list(prediction)[0][0] >= 0.95:
            cv2.putText(frame, 'Despierto', (10, int(height*0.08)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
            contador -= 1

        # Control del contador y reproducción del sonido de alarma
        if contador >= 15:
            contador = 15
            p.play()  # reproducir alarma si está dormido
        elif 0 < contador < 15:
            p.stop()  # detener alarma si se despierta
        elif contador < 0:
            contador = 0

    # Mostrar el video de la webcam
    cv2.imshow('video', frame)

     #Tecla de salida - acaba la transmisión
    if cv2.waitKey(1) & 0xFF == ord('q'):
        video_capture.release()
        cv2.destroyAllWindows()
        break


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 38ms/step
